In [1]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import feedparser
import os
import numpy as np
from requests import Session
from multiprocessing.dummy import Pool as ThreadPool

In [2]:
# Getting the current work directory (cwd)
thisdir = os.getcwd()
snapshots = []

# r=root, d=directories, f = files
for r, d, f in os.walk(thisdir+"/vn"):
    for file in f:
        if file.endswith(".snapshot"):
            snapshots.append(os.path.join(r, file))

In [4]:
data = pd.DataFrame([])
for snap in snapshots:
    parser = feedparser.parse(snap)
    for post in parser.entries:
        try:
            title = post.title
            link = post.link
            date = post.published
            data = data.append(pd.DataFrame({'title': title, 'url': link,'date' : date}, index=[0]), ignore_index=True)
        except:
            continue
    

In [18]:
data = data.drop_duplicates()

In [19]:
data.shape

(649, 3)

In [17]:
def categorize(x):
    if 'vietnamese' in x:
        return "vietnam"
    elif 'business' in x:
        return "economy"
    elif 'world' in x:
        return "world"
    elif 'sports' in x:
        return "sports"
    else:
        return np.nan


In [20]:
data['category'] = data["url"].apply (lambda x: categorize(x))

In [21]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}
# Add headers
s.headers.update(headers)

    
def get_user_need(page):
    try:
        article = s.get(page)
        soup = BeautifulSoup(article.content, 'lxml')
        x = [i for i in soup.find("script",text = re.compile(r'(?<="campaignName".").+?(?=\")'))][0]
        return re.search(r'(?<="campaignName".").+?(?=\")',x)[0]
    except:
        return np.nan
    
def get_text(url):
    try:
        article = s.get(url)
        soup = BeautifulSoup(article.content, 'lxml')
        body = soup.find(role='main')
        text = [p.text for p in body.find_all('p',dir="ltr")]
        return ''.join(text)
    except:
        return np.nan


In [22]:
%%time
pool = ThreadPool(5)
user_need = pool.map(get_user_need, data["url"])
data["User Need"] = user_need

CPU times: user 19.9 s, sys: 1.43 s, total: 21.3 s
Wall time: 1min 21s


In [23]:
%%time
page_text = pool.map(get_text, data["url"])
data["text"] = page_text

CPU times: user 10.8 s, sys: 1.33 s, total: 12.1 s
Wall time: 16.4 s


In [25]:
data["User Need"] = data["User Need"].apply(lambda s: re.sub(r"(WS - )", "", s) if pd.notnull(s) else s)

In [26]:
data.shape

(649, 6)

In [29]:
data2=pd.read_csv("bbc_vietnam.csv")

In [30]:
data2.shape

(1752, 6)

In [32]:
data = data.append(data2,ignore_index=True)

In [34]:
data = data.drop_duplicates(subset="url")

In [35]:
data.shape

(2401, 6)

In [36]:
data["User Need"].value_counts()

Update me              1425
Give me perspective     617
Keep me on trend        140
Divert me                70
Educate me               69
Inspire me               48
CS                       20
News - Disclosure         1
Name: User Need, dtype: int64

In [37]:
data.to_csv("BBC_vietnam_final.csv",index=False)